In [ ]:
# Train IMDb Classifier
# 还没进行 ， 我们小组决定再微调训练一个专门用于IMDB情感分类的分类器
# 就选最轻量级的Distilbert
# 快疯了，实在不行 ,换镜像
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [ ]:
# Load IMDb dataset
from datasets import load_dataset, load_metric
ds = load_dataset("imdb")
ds

In [ ]:
ds['train'].features

In [ ]:
# Load Pretrained DistilBERT
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Prepocess Data
def tokenize(examples):
    outputs = tokenizer(examples['text'], truncation=True)
    return outputs

tokenized_ds = ds.map(tokenize, batched=True)

In [ ]:
# Prepare Trainer

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(num_train_epochs=1,
                                  output_dir="distilbert-imdb",
                                  push_to_hub=True,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  evaluation_strategy="epoch")

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
trainer = Trainer(model=model, tokenizer=tokenizer,
                  data_collator=data_collator,
                  args=training_args,
                  train_dataset=tokenized_ds["train"],
                  eval_dataset=tokenized_ds["test"], 
                  compute_metrics=compute_metrics)

In [ ]:
# Train Model and Push to Hub

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()